Import the necessary packages

In [1]:
import cv2
import numpy as np
import os
import platform
if platform.uname().system == 'Windows':
    os.chdir('C:/Users/ralt6/source/repos/ElegansRec/')
else:
    os.chdir('/home/ryan/Projects/ElegansRec/')

Declare image paths

In [2]:
main_img_path = 'src/images/Elegans_Test_1.jpg'
el_img_path = 'src/images/ET1_sub.jpg'

Load images and reduce size by half

In [3]:
main_img = cv2.imread(main_img_path, cv2.IMREAD_REDUCED_COLOR_2)
el_img = cv2.imread(el_img_path, cv2.IMREAD_REDUCED_COLOR_2)

Display image sizes

In [4]:
h, w = main_img.shape[:2]
print("Height = {}, width = {}".format(h,w))
h,w = el_img.shape[:2]
print("Height = {}, width = {}".format(h,w))

Height = 839, width = 839
Height = 96, width = 157


Extract R,B,G values from a particular pixel

In [5]:
b, g, r = main_img[100,100]
print('R: {}, G: {}, B: {}'.format(r,g,b))

R: 5, G: 5, B: 5


Extract a particular region of interest

In [6]:
roi = main_img[100:400, 100:400]
def display_image(img):
    cv2.imshow('Section', img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

Convert images to grayscale. Set pixel values above 120 to 255 and pixel at 120 or below to 0. Display image

In [7]:
main_gs = cv2.cvtColor(main_img, cv2.COLOR_BGR2GRAY)
enemy_gs = cv2.cvtColor(el_img, cv2.COLOR_BGR2GRAY)
ret, thresh1 = cv2.threshold(main_gs, 120, 255, cv2.THRESH_BINARY)
display_image(thresh1)

Compare enemy image against test image


In [8]:
result = cv2.matchTemplate(main_img, el_img, cv2.TM_CCOEFF_NORMED)
display_image(result)

Get the location of the max item observed

In [9]:
min_val, max_val, min_loc, max_loc = cv2.minMaxLoc(result)
print("Image located at Location: {} with value: {}".format(max_loc, max_val))

Image located at Location: (413, 468) with value: 0.988216757774353


Draw rectangle around found dead enemy in test image

In [10]:
eh,ew = el_img.shape[:2]
top_left = max_loc
bottom_right = (max_loc[0] + ew, max_loc[1] + eh)
result = cv2.rectangle(main_img, top_left, bottom_right, color=(0,255,0), thickness=2, lineType=cv2.LINE_4)
display_image(result)
cv2.imwrite('src/images/test_image_1_ti1_result.jpg', result)

True

Gather list of results of nearly matching images above a certain threshold

In [11]:
main_img = cv2.imread(main_img_path, cv2.IMREAD_REDUCED_COLOR_2)
result = cv2.matchTemplate(main_img, el_img, cv2.TM_CCOEFF_NORMED)
def find_above_threshold(result, threshold):
    locations = np.where(result >= threshold)
    return list(zip(*locations[::-1]))
    #for i in range(len(result)):
    #    for j in range(len(result[i])):
    #        if result[i][j] > threshold:
    #            print('Print threshold found at {} with value {}'.format((i,j), result[i][j]))
print(find_above_threshold(result, 0.4))

[(427, 459), (428, 459), (429, 459), (430, 459), (431, 459), (424, 460), (425, 460), (426, 460), (427, 460), (428, 460), (429, 460), (430, 460), (431, 460), (420, 461), (421, 461), (422, 461), (423, 461), (424, 461), (425, 461), (426, 461), (427, 461), (428, 461), (429, 461), (430, 461), (431, 461), (416, 462), (417, 462), (418, 462), (419, 462), (420, 462), (421, 462), (422, 462), (423, 462), (424, 462), (425, 462), (426, 462), (427, 462), (428, 462), (429, 462), (430, 462), (431, 462), (413, 463), (414, 463), (415, 463), (416, 463), (417, 463), (418, 463), (419, 463), (420, 463), (421, 463), (422, 463), (423, 463), (424, 463), (425, 463), (426, 463), (427, 463), (428, 463), (429, 463), (430, 463), (410, 464), (411, 464), (412, 464), (413, 464), (414, 464), (415, 464), (416, 464), (417, 464), (418, 464), (419, 464), (420, 464), (421, 464), (422, 464), (423, 464), (424, 464), (425, 464), (426, 464), (427, 464), (428, 464), (429, 464), (408, 465), (409, 465), (410, 465), (411, 465), (41

Draw rectangles for all found locations

In [12]:
def draw_rectangles(img, locations, dimension, name):
    for i in locations:
        top_left = i
        bottom_right = (top_left[0] + dimension[0], top_left[1] + dimension[1])
        cv2.rectangle(img, top_left, bottom_right, color=(0,255,0), thickness=2, lineType=cv2.LINE_4)
    cv2.imwrite('src/images/{}.jpg'.format(name), img)

el_img = cv2.imread(el_img_path)
eh,ew = el_img.shape[:2]
methods = [(cv2.TM_SQDIFF, 'SQDIFF'), (cv2.TM_SQDIFF_NORMED, 'SQDIFF_NORMED'), (cv2.TM_CCORR, 'CCORR'), 
    (cv2.TM_CCORR_NORMED, 'CCORR_NORMED'), (cv2.TM_CCOEFF, 'CCOEFF'), (cv2.TM_CCOEFF_NORMED, 'CCOEFF_NORMED')]
for method in methods:
    main_img = cv2.imread(main_img_path)
    result = cv2.matchTemplate(main_img, el_img, method[0])
    locations = find_above_threshold(result, 0.8)
    draw_rectangles(main_img, locations, (ew,eh), 'ti1_el_test_result2_{}'.format(method[1]))

In [19]:
main_img = cv2.imread(main_img_path)
result = cv2.matchTemplate(main_img, el_img, cv2.TM_CCOEFF_NORMED)
locations = find_above_threshold(result, 0.2)
draw_rectangles(main_img, locations, (ew,eh), 'ti1_el_test_result2_{}'.format('CCOEFF_NORMED_0.2'))